# Manhattan Rental Apartments Clustering

### Xian Lai
xlai4@fordham.edu   
Apr.2017


=======================================================
<img src="images/cover_image.png" width="1200">

### Abstract:
A city functions like a gigantic sophisticate network. Within it buildings and blocks are connected by visible urban infrastructure systems and invisible functional dependencies. On the other hand, the difference of locations and functionality also divides the city into many regions. Under different fucntionality, the boundaries of these regions are different. For political administration, we have boroughs, community districts and neighbourhoods, for postal service, we have zip codes, etc. 

In this projet, I made use of rental apartment online listing dataset and New York building footprint dataset to explore the possible geographic boundaries or patterns of apartment rental market. Equivalent to finding boundaries, clustering are performed to find the best grouping of buildings with respect to their location and rental market popularity. Then we show how different properties like bedroom number, is there elevator in building, is there fitness center in building etc. can affect the clustering patterns.

Particularly, we are going to:
- Interpolate the popularity of every building in the building dataset.
- Select the best clustering model using a sample dataset.
- Perform the final model on the full dataset with different conditions we are interested in.

In [1]:
from helpers import *
output_notebook()

Loading BokehJS ...

## I. Listings Data
  
The listing dataset contains about 46,000 rental apartment online listings in Manhattan. Each listing containing information about rental apartments’ geographic location, popularity (defined by how many visits of listing webpage) and some other description features like facilities, number of bedrooms, bathrooms, rental price, etc.

In [2]:
listings_path = 'data/listings_preprocessed.pkl'
blocks_path   = 'data/block_centroids.pkl'
polygons_path = 'data/block_polygons_webMerc.geojson'

listings = pd.read_pickle(listings_path).reset_index(drop=True)
centroids = pd.read_pickle(blocks_path).reset_index(drop=True)
polygons = open(polygons_path).read()

In [3]:
listings.head()

,bathrooms,bedrooms,listing_time,features,interest_level,y,x,price,centroid
0,0.15,0.375,0.937237,,0.5,4.970321e+06,-8.231241e+06,0.000659,"(-8231241.44798163, 4970321.247199517)"
1,0.10,0.250,0.804684,doormanelevatorfitnesscentercatsalloweddogsall...,0.0,4.982107e+06,-8.233935e+06,0.001208,"(-8233935.379658829, 4982106.959103675)"
2,0.10,0.125,0.170960,laundryinbuildingdishwasherhardwoodfloorspetsa...,1.0,4.973891e+06,-8.237843e+06,0.000625,"(-8237842.693785673, 4973890.728889039)"
3,0.10,0.125,0.181733,hardwoodfloorsnofee,0.0,4.976109e+06,-8.234047e+06,0.000720,"(-8234046.699149621, 4976109.458764397)"
4,0.10,0.500,0.293677,pre-war,0.0,4.986431e+06,-8.231998e+06,0.000737,"(-8231998.420519025, 4986430.972419331)"


First of all, building or block is a better unit for clustering rather than apartment. Second, as the following figure shows, the listed apartments are dense in some popular areas, but sparse in others. To get a more comprehensive understanding of all neighborhoods in Manhattan. I immigrant the information from listings dataset to blocks dataset by interpolation.

In [ ]:
# plot showing where the listings located in New York
p = base_plot(
    plot_width=900, plot_height=1000, x_range=(-8240000,-8220000), 
    y_range=(4970000,4990000), axis_visible=False
)
p = plot_patches(p, polygons, alpha=0.5, silent=True)
plot_scatter(p, listings['x'], listings['y'], marker='o', size=3, alpha=0.1)

![](images/listings.png)

## II. Model Setup and Selection

### interpolation
Assuming the popularity of rental apartment is geographically continuous, namely the popularity of one building is similar to surrounding buildings, I can interpolate the popularities of every building using the information from listing dataset. Then clustering will be performed on building dataset instead.

The popularity of a building is calculated from surrounding listed apartments based on the distance in space and time.

Assuming y is our target building, ${x_0, x_1, ..., x_n}$ is the set of nearby apartments. $u(x_i)$ is the popularity of $x$. We have the calculation formula:

If $d(y,x_i) \neq 0$ for all $x_i$:
$$u(y)=\frac{\sum_{i=1}^{n} w(x_i)*u(x_i)}{\sum_{i=1}^{n} w(x_i)}$$
where 
$$w(x_i)=\frac{1}{d(y,x_i)^p \times t}$$
If $d(y,x_i) = 0$ for some $x_i$:
$$u(y)=u(x_i)$$

### clustering
Then hierarchical clustering is performed on interpolated block dataset using their longitude, latitude and the popularity. The hierarchical clustering allows us to choose geographic granularity easily, so we gain different points of view from larger areas to small neighbourhoods.

For the clustering model, there are 2 hyperparameters to decide:
- **method**: Method to calculate distance between clusters.
- **metric**: Metric to calculate distance between data points.

Until now, we have 4 hyperparameters for our model: 
- radius : {0.005, 0.01, 0.05}
- IDWpower : {0.1, 0.5, 1.0, 3.0}
- method : {'average', 'weighted', 'complete', 'centroid'}
- metric : {'cityblock', 'euclidean'}

### Evalution:
To select the best one from these 84 models, we use the following 6 criteria:
1. n_singlton : The number of singleton clusters.
2. smClusterSize: The cluster size at the 15th percentile ranking from small to big.
3. lgClusterSize: The cluster size at the 85th percentile ranking from small to big.
4. lgClusterArea: The cluster area at the 85th percentile ranking from small to big.
5. interVariance: The within cluster popularity variance.
6. intraVariance: The between cluster popularity variance.

The first 4 criteria evaluate whether a model yields balanced clustering with respect to both size and area. For this application, we don't want the clustering with a few large clusters and many small clusters.

The last 2 criteria evaluate whether a model put nearby buildings with similar popularity in the same cluster and the ones with different popularity into different clusters.

In [ ]:
listings = pd.read_pickle("data/listings_preprocessed_coor.pkl")
centroids = pd.read_pickle("data/block_centroids_coor.pkl")

In [5]:
params_interpolation = list(ParameterGrid({
    'rad':[0.005, 0.01, 0.05],
    'IDWpower':[0.1, 0.5, 1.0, 3.0],
}))

params_clustering = list(ParameterGrid({
    'method':['average', 'weighted', 'complete'],
    'metric':['cityblock', 'euclidean'],
})).append({
    'method':'centroid',
    'metric':'euclidean'
})

def grid_search(listings):
    """ perform grid searching on sampled listings with all parameter combinations.
    """
    # In total there are 2 type of hyperparameters and 12 combinations
    # for interpolation. So we use a list HCs to save all 12 possible
    # hierarchical clusterings.
    ics = []
    for param in params_interpolation:
        ic = IC(listings, blocks)
        ic.prepare_data(**param)
        ics.append((param.copy(), ic))

    print("\tInterpolation models built.")

    # There are again 2 types of hyperparameters and in total 7 
    # combinations for clustering. We will apply each of them onto
    # each of the interpolation clustering instances produced above.
    stats = []
    for param in params_clustering:
        for param_, ic in ics:
            param_.update(param)
            ic.clustering(n_clusters=300, **param)
            stat = ic.clusteringStats
            stat.update({'param':param_.copy()})
            stats.append(stat)

    print("\tClustering models built.")

    return {'model':ics, 'stats':stats}

### Selection

#### Grid search
In the model testing, for each hyper-parameter combination, we performed clustering 100 times repetitively. In each repetition, the popularities of blocks were interpolated from 5000 listings sampled from the whole dataset, and the clustering stats were recorded. Here we load the clustering stats for comparison and final selection.

In [6]:
def model_on_samples(x):
    """
    """
    print("excuting process: ", x)
    listings_sub = listings_all.sample(n=5000)
    return grid_search(listings_sub)

# pool    = mp.Pool(processes=3)
# results = pool.map(model_on_samples, range(100))

# with open("data/models.pkl", "wb") as f:
#     pickle.dump([x['model'] for x in results], f)
# with open("data/stats.pkl", "wb") as f:
#     pickle.dump([x['stats'] for x in results], f)
    
load_stats = lambda x: pd.DataFrame(x).set_index('param')
with open('data/stats.pkl', 'rb') as f:
    clustering_stats = [load_stats(_) for _ in pickle.load(f)]
    clustering_stats = [scale_by_col(_) for _ in clustering_stats]
clustering_stats[0].head()

,interVariance,intraVariance,lgClusterArea,lgClusterSize,n_singlton,smClusterSize
param,,,,,,
"{'rad': 0.005, 'method': 'average', 'IDWpower': 0.1, 'metric': 'cityblock'}",0.484296,0.297020,0.557066,0.619772,0.590909,0.0
"{'rad': 0.01, 'method': 'average', 'IDWpower': 0.1, 'metric': 'cityblock'}",0.786169,0.599194,0.687356,0.315589,0.336364,0.0
"{'rad': 0.05, 'method': 'average', 'IDWpower': 0.1, 'metric': 'cityblock'}",0.996380,0.848915,0.859877,0.087452,0.136364,0.0
"{'rad': 0.005, 'method': 'average', 'IDWpower': 0.5, 'metric': 'cityblock'}",0.392411,0.282556,0.618887,0.619772,0.590909,0.0
"{'rad': 0.01, 'method': 'average', 'IDWpower': 0.5, 'metric': 'cityblock'}",0.756530,0.593802,0.703830,0.391635,0.354545,0.0


#### Investigate the criterion behavior

Gaining the evaluation decisions given by 6 criteria for each clustering model, we need to find a way to combine them. There exist several different ways of combining the outputs of the scoring systems, including score combination, rank combination, voting, average combination, weighted combination etc. 

Here I use a scoring systems combination method introduced by Hsu and Taksa*:  
Let $S_m(n)$ and $R_m(n)$ be the score and rank given by $m^{th}$ criterion on $n^{th}$ model respectively. We will have $S_m(n) \in [0,1]$ with highest scoring = 1 and $R_m(n) \in [1,N]$ with highest ranking = 1. Then we can investigate the scoring behavior of different criterions defined by Rank-Score Characteristic(RSC):
$$RSC_m(n) = \frac{S_m(n)}{R_m(n)}$$

The RSC curves of each criterion will form rank-score graph that tells us how different each criterion deciding their scoring. The following picture is an illustration of 3 scoring systems. The scoring system who assigns scores in a linearly decreasing fashion will have a linear rank-score curve like $f_2$ does. The system who habitually assigns high scores to a large subset of its top ranked candidates will have a graph that is not a straight line, but has a low slope for the top ranked candidates and a higher slope for the remainder similar to $f_4$. A third class of scoring behavior is exemplified by $f_1$. In this case, the expert habitually gives higher scores to a small subset of its top ranked candidates and much lower scores to the rest. 

![](images/RSC_graph.png)

Hsu and Taksa indicate that a diversity measure based on the rank-score graph can be used to determine whether a score or rank fusion will produce a better result. **When the rank-score graphs of two systems are very SIMILAR, then a Score Combination will produce the best fusion. When the rank-score graphs are very DIFFERENT, then a Rank Combination produces the better result.**

\* Hsu, D.F. and Taksa, I., Comparing rank and score combination methods for data fusion in information retrieval.

In [ ]:
plot_rsg(clustering_stats[0], figure_size=600)

![](images/rsg.png)

In [ ]:
compare_rsg(clustering_stats[:16], 4)

![](images/rsgs.png)

#### Combine criteria

As we plot the graph with ranking as x-axis and scoring as y-axis, we see that all 6 scoring systems have similar behavior pattern. They all prefer to give high scores to high rankings and give low scores to low rankings. That means they are all confident about their judging. And the confidence can be measured by the area under curve(AUC). The smaller AUC is, the more confident this criterion is.

In [12]:
calc_auc = lambda grp: grp.apply(score_func, axis=0).sum(axis=0)
df_auc = pd.DataFrame([calc_auc(grp) for grp in clustering_stats])
compare_auc(df_auc, fw=800, fh=400, title="AUC for 100 models")

Here we see the mean AUC of some criteria are higher than the others. And some of them are more stable(small variance) than the others. Since mean and variance have different range, we minmax scale them first and then calculate the weighting for score combination using the fomulation:

$$
weight_i = \frac{1}{(var_i + 1) * (mean_i + 1)} \\
\text{weight_norm}_i = \frac{weight_i}{\sum_i weight_i} \\
\text{combined_score} = \sum_i {\text{weight_norm}_i * \text{scaled_score}_i}
$$

In [26]:
def combine_score(group, weightings, n_show=5):
    combined_score = group.apply(score_func, axis=0).dot(weightings)
    if n_show is not None:
        print(combined_score.head(n=n_show))
    return combined_score

weightings = 1 / ((score_func(df_auc.var()) + 1) * (score_func(df_auc.mean()) + 1))
weightings = weightings / weightings.sum()
df_performance = pd.concat([combine_score(grp, weightings, n_show=None) for grp in clustering_stats], axis=1)

df_performance.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
param,,,,,,,,,,,,,,,,,,,,,
"{'rad': 0.005, 'method': 'average', 'IDWpower': 0.1, 'metric': 'cityblock'}",0.397294,0.385124,0.407795,0.376349,0.391851,0.396146,0.383441,0.399501,0.400547,0.514039,...,0.396091,0.394612,0.447041,0.352579,0.393624,0.395934,0.394971,0.432208,0.386579,0.361446
"{'rad': 0.01, 'method': 'average', 'IDWpower': 0.1, 'metric': 'cityblock'}",0.461451,0.473224,0.486861,0.477286,0.485870,0.459550,0.456661,0.474271,0.490695,0.494874,...,0.482269,0.465401,0.476259,0.454433,0.491993,0.493475,0.478938,0.473154,0.476935,0.481392
"{'rad': 0.05, 'method': 'average', 'IDWpower': 0.1, 'metric': 'cityblock'}",0.520980,0.552633,0.545383,0.562787,0.537408,0.543494,0.566087,0.538183,0.522236,0.530258,...,0.532175,0.548908,0.564591,0.542131,0.555463,0.569162,0.527691,0.535149,0.513133,0.528462
"{'rad': 0.005, 'method': 'average', 'IDWpower': 0.5, 'metric': 'cityblock'}",0.386405,0.396510,0.420789,0.368996,0.377645,0.376669,0.391171,0.390988,0.402560,0.398852,...,0.387413,0.389527,0.388616,0.414467,0.390001,0.389966,0.396173,0.493938,0.392261,0.356512
"{'rad': 0.01, 'method': 'average', 'IDWpower': 0.5, 'metric': 'cityblock'}",0.467681,0.475165,0.474443,0.480048,0.466801,0.458116,0.481053,0.453796,0.472047,0.493656,...,0.463381,0.450054,0.478708,0.446889,0.498714,0.490452,0.503078,0.493779,0.473665,0.459007


After sorting by the mean combined score on 100 samplings, we can clearly see some models outperform the others. 

In [22]:
df_performance['mean_score'] = df_performance.sum(axis=1)
df_performance = df_performance.sort_values('mean_score', ascending=True)
df_performance = df_performance.drop('mean_score', axis=1)
params = list(df_performance.index)

In [23]:
plot_image(df_performance.values, title="Clustering Performance", px_w=8, px_h=8)

In [25]:
params[-5:]

[{'rad': 0.05, 'method': 'complete', 'IDWpower': 0.5, 'metric': 'cityblock'},
 {'rad': 0.05, 'method': 'complete', 'IDWpower': 1.0, 'metric': 'euclidean'},
 {'rad': 0.05, 'method': 'complete', 'IDWpower': 0.5, 'metric': 'euclidean'},
 {'rad': 0.05, 'method': 'complete', 'IDWpower': 0.1, 'metric': 'euclidean'},
 {'rad': 0.05, 'method': 'complete', 'IDWpower': 1.0, 'metric': 'cityblock'}]

## III. Clustering and Visualization

With the final parameters we picked, we can perform clustering on full building dataset and visualize the result.

In [6]:
listings = pd.read_pickle("data/listings_preprocessed_coor.pkl").sample(n=300)
centroids = pd.read_pickle("data/block_centroids_coor.pkl")

In [7]:
def query(bool_mask=None, color='mean', title=None):
    """
    """
    param_final = {'rad':0.05, 'IDWpower':1.0, 'method':'complete', 'metric':'cityblock'}
    if bool_mask is None:
        subset = listings.copy()
    else:
        subset = listings[bool_mask].copy()
    ic = IC(subset, centroids)
    ic.analysis(**param_final)
    
    v = vis.Visual(figWidth=300, title=title)
    v.plotClustering(ic.clusters, show=True)
    return ic

In [ ]:
ic = query(bool_mask=None, title=None, color='mean')

<img src="images/all.png" width="400">


### 1. Color the clusters by their statistics

In the process of clustering, we calculate the following statistics for each cluster:

- Popularity mean  
- Popularity variance  
- cluster size  
- cluster area  

We can either use them to filter clusters, (For example, filter out 100 clusters with highest popularities.) or use them as color coding to visualize these clusters. (For example, plot the clusters colored by their popularity mean.)

In [ ]:
largeClusters_200 = ic.clusterStats\
    .copy()\
    .sort_values('area', ascending=False)\
    .head(n=200)
data = ic.clusters.copy()
mask = data['cluster'].isin(largeClusters_200.index)
data = data[mask]

In [ ]:
v1 = vis.Visual(figWidth=300, title='colored by mean')
v1.plotClustering(data, color='mean', show=False)
v2 = vis.Visual(figWidth=300, title='colored by variance', 
     x_range=v1.p.x_range, y_range=v1.p.y_range)
v2.plotClustering(data, color='variance', show=False)
v3 = vis.Visual(figWidth=300, title='colored by size', 
     x_range=v1.p.x_range, y_range=v1.p.y_range)
v3.plotClustering(data, color='size', show=False)
# show(row(v1.p, v2.p, v3.p))

<img src="images/top_area_clusters.png" width="1300">

### 2. Query clustering using different building properties

Since the listing dataset contains information about building properties like price, fitness centers, bedroom numbers etc, we can produce different subset of listing data and interpolate the building popularity from this subset and hence get a different clustering. (For example, if we want to compare the clustering of high-price rentals to that of low-price rentals, we can create 2 subsets, get 2 clusterings and compare the difference in final plottings.)

As examples, we will compare the following clusters:
    - low price rentals vs high price rentals
    - 1-bedroom rentals, 2-bedroom rentals vs 4-bedroom rentals
    - rentals without elevators vs rentals with elevators

In [127]:
listings.head()

,bathrooms,bedrooms,listing_time,features,interest_level,y,x,price,centroid
933,0.1,0.125,0.744731,doormanelevatorcatsallowed,0.5,40.7358,-73.9859,0.000769,"(-73.9859, 40.73579999999999)"
14677,0.1,0.000,0.744731,doormanelevatorfitnesscenterpre-warlaundryinbu...,0.0,40.7301,-73.9927,0.000692,"(-73.9927, 40.7301)"
28029,0.1,0.375,0.361593,elevatorhardwoodfloorsdogsallowedcatsallowed,0.0,40.7598,-73.9924,0.000837,"(-73.9924, 40.75979999999999)"
13480,0.1,0.000,0.710539,hardwoodfloors,0.0,40.7832,-73.9482,0.000402,"(-73.9482, 40.7832)"
22650,0.3,0.375,0.936768,elevatorpre-warlaundryinbuildingnofee,0.0,40.7382,-73.9918,0.003052,"(-73.99179999999998, 40.73819999999999)"


### Price levels:

In [55]:
listings['price'].describe()

count    49272.000000
mean         0.000844
std          0.004919
min          0.000000
25%          0.000547
50%          0.000692
75%          0.000904
max          1.000000
Name: price, dtype: float64

In [ ]:
query(bool_mask=(listings['price'] > 0) & (listings['price'] < 0.000547))
query(bool_mask=(listings['price'] > 0.000547) & (listings['price'] < 0.000692))
query(bool_mask=(listings['price'] > 0.000692) & (listings['price'] < 0.000904))

<img src="images/prices.png" width="1300">

### Number of bedrooms:

In [11]:
listings['bedrooms'].describe()

count    4000.000000
mean        0.192031
std         0.140729
min         0.000000
25%         0.125000
50%         0.125000
75%         0.250000
max         0.750000
Name: bedrooms, dtype: float64

In [ ]:
query(bool_mask=(listings['bedrooms'] == 0.125000))
query(bool_mask=(listings['bedrooms'] == 0.250000))
query(bool_mask=(listings['bedrooms'] == 0.750000))

<img src="images/bedrooms.png" width="1300">


### Elevator

In [ ]:
query(bool_mask=(listings['features'].apply(lambda x: 'elevator' in x)))
query(bool_mask=(listings['features'].apply(lambda x: 'elevator' not in x)))

<img src="images/elevators.png" width="600">